8 PUZZLE Using Greedy Best First Search.

Here we have defined manhattan distance as Heuristic function

In [10]:
import numpy as np
import time    
from copy import deepcopy

Here, I have defined the function for taking input.

In [11]:
def getting_states():
    string = input('Enter the Elements serially for the 8 PUZZLE: ')
    return string

Here, This is inversion method for checking whether the problem is solvable or not.
Suppose i am taking an example of 1,2,3,4,5,6,8,7,0. In this case 1,2,3,4,5,6,7 is having 0 inversion but 8 is having 1 inversion as it's preceding '7'.
Hence, the given problem is 'Not Solvable'.
consider one more example : 2,1,3,4,5,6,8,7,0.
In this case we have total number of inversions =2. 2 is having one inversion and 8 is having one inversion.
so, Toatl number of inversion = 2.(i.e. Even number of inversions)
we can conclude that if a problem having odd number of inversions then it is Not solvable and if a problem has even number of inversions then problem can be solved.


In [12]:
def inversion_method(state):
    i = state[state != 0]
    inv = 0
    for j in range(len(i) - 1):
        b = np.array(np.where(i[j+1:] < i[j])).reshape(-1)
        
        inv += len(b)
    return inv

Here, I have defined a function for heuristic calculation based on Manhattan distance formula for each element(i.e.for each element, by how much distance they are lagging in the goal state) and then i am adding distance of each elements for our heuristic at that state.

In [13]:
def cal_heuristic(s, g):
    distance = 0
    n = len(s)
    for i in range(n):
        for j in range(n):
            s_Row = int(i/3)
            s_Column = i%3
            g_Row = int(j/3)
            g_Column = j%3
        distance += abs(s_Row-g_Row) + abs(s_Column-g_Column)
    return distance

This function is checking for the occurence of each elements. i.e checking user input for correctness

In [14]:

def all(s):
    set = '012345678'
    return 0 not in [c in s for c in set]



This function is generating optimal by taking state as argument returned by the solve function.

In [15]:
def generate_optimal(state):
    optimal = np.array([], int).reshape(-1, 9)
    last = len(state) - 1
    while last != -1:
        optimal = np.insert(optimal, 0, state[last]['board'], 0)
        last = int(state[last]['parent'])
    return optimal.reshape(-1, 3, 3)



In [16]:
def solve(board, goal):                                    #solves the board
    moves = np.array(   [   ('u', [0, 1, 2], -3),
                            ('d', [6, 7, 8],  3),           #Defined the moves as , u - up, d - down, l- left and r - right.
                            ('l', [0, 3, 6], -1),
                            ('r', [2, 5, 8],  1)
                            ],
                dtype=  [  ('move',  str, 1),
                           ('pos',   list),
                           ('delta', int)
                           ]
                        )

    dtstate = [ ('board',  list),
                ('parent', int),
                ('hn',     int)
                ]
    #values of initial state
    parent = -1
    gn     = 0
    hn     = cal_heuristic(board, goal)
    state = np.array([(board, parent,hn)], dtstate)
    
    #initialization of priority queue
    dtpriority = [  ('pos', int),      
                    ('hn', int)
                    ]

    priority = np.array( [(0, hn)], dtpriority)
    
    while True:
        priority = np.sort(priority, kind='mergesort', order=['hn', 'pos']) #sorting the priority queue
        pos, hn = priority[0]                                        #taking out first from sorted queue
        priority = np.delete(priority, 0, 0)                         #remove from queue what we are exploring
        board, parent,hn = state[pos]
        board = np.array(board)
        loc = int(np.where(board == 0)[0])                           #Locating Blank('0')
        hn = hn + 1                            

        for m in moves:
            if loc not in m['pos']:
                succ = deepcopy(board)                                 # generating new state as copy of current
                succ[loc], succ[loc + m['delta']] = succ[loc + m['delta']], succ[loc]   

                if ~(np.all(list(state['board']) == succ, 1)).any():   # checking if new (not repeat)
                    hn = cal_heuristic(succ, goal)                     #calculating manhattan distance    
                    q = np.array(   [(succ, pos,hn)], dtstate)         #generate and add new state in the list
                    state = np.append(state, q, 0)
                                                        
                    q = np.array([(len(state) - 1, hn)], dtpriority)    
                    priority = np.append(priority, q, 0)

                    if np.array_equal(succ, goal):                     
                        print('Goal achieved!')                       #checking is this our goal state or not.
                        return state, len(priority)

    return state, len(priority)
    end = time.time()
    print(f"Total runtime of the program is {end - begin}")   #total runtime of the program is being calculated


In [17]:
def main():
    print("Please enter the starting state of 8 PUZZLE ")  #taking initial input from user.
    init_mat = getting_states()
    
    if len(init_mat) != 9 or all(init_mat) == 0:  #checking that input is valid or not
        print('incorrect Input State')
        return
    
    print("Please enter the goal state of 8 PUZZLE")  #taking goal input from user.
    goal_mat = getting_states()
    if len(goal_mat) != 9 or all(goal_mat) == 0:  #checking that goal input is valid or not.
        print('incorrect Output State')
        return
    
    begin = time.time()
    if(init_mat==goal_mat):
            print(init_mat)                       #if initial state = goal state then print solution found.
            print("solution found")
            return
        
    init_state = np.array(list(map(int, init_mat)))  #taking input as array
    goal_state = np.array(list(map(int, goal_mat)))  #taking goal as array
    
    if (inversion_method(init_state) % 2 != 0):      #checking for inversions.
        print('not solvable')
        return
    
            
    state, explored = solve(init_state, goal_state)
    print()
    print('Total number of states generated:', len(state))
    print('Total number of states explored: ', len(state) - explored)
    print()
    #Generating optimised steps
    optimal = generate_optimal(state)
    print('Total optimized steps:', len(optimal) - 1)
    print()
    print(optimal)
    print()
    end = time.time()
    print(f"Total runtime of the program is {end - begin}")   #total runtime of the program is being calculated

In [18]:
if __name__ == "__main__":
    main()                                                  #calling main function

Please enter the starting state of 8 PUZZLE 


Enter the Elements serially for the 8 PUZZLE:  123406758


Please enter the goal state of 8 PUZZLE


Enter the Elements serially for the 8 PUZZLE:  123456780


Goal achieved!

Total number of states generated: 9
Total number of states explored:  3

Total optimized steps: 2

[[[1 2 3]
  [4 0 6]
  [7 5 8]]

 [[1 2 3]
  [4 5 6]
  [7 0 8]]

 [[1 2 3]
  [4 5 6]
  [7 8 0]]]

Total runtime of the program is 0.009000301361083984
